In [15]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import re

In [16]:
# --- NLTK Setup (Ensure these are downloaded if needed) ---
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# --- Configuration ---
# NOTE: Replace the INPUT_FILE with your exact local path for execution!
INPUT_FILE = "raw_data_essay_set1.xlsx"
OUTPUT_FILE = "essay_dataset.xlsx"
SCORE_FOR_COMMON_WORDS = 12 # Used to define the common features


In [17]:
# Initialize NLTK components
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# Create a translation table to remove punctuation efficiently (similar to your reference code)
translator = str.maketrans('', '', string.punctuation)


In [18]:
def preprocess_essay(text):
    """
    Combines all preprocessing steps based on the reference code logic, 
    including case handling, punctuation removal, stopword removal, 
    and special token grouping.
    """
    if pd.isna(text) or not text:
        return []
    
    raw_tokens = str(text).split() # 1. Split into tokens first
    
    # 2. Selective Lowercasing (Keep @-words' original case)
    cased_tokens = [word if word.startswith('@') else word.lower() for word in raw_tokens]
    
    # 3 & 4. Join for Punctuation Removal
    joined_string = ' '.join(cased_tokens)
    clean_string = joined_string.translate(translator)
    
    # 5. Split into clean tokens (after punctuation removal)
    clean_tokens = clean_string.split()
    
    processed_tokens = []
    
    for word in clean_tokens:
        
        # Determine if the token is a potential category token
        # (Uppercase letters followed by optional digits: e.g., CAPS1, LOCATION)
        # We check the case of the *current* word, which will be uppercase 
        # only if it was an @-word in the original text (e.g. @CAPS1 -> CAPS1).
        category_match = re.match(r'([A-Z]+)\d*$', word)
        
        if category_match:
            # If it matches, extract the base category (e.g., CAPS1 -> CAPS)
            lemmatized_word = category_match.group(1)
        else:
            # If it's a regular lowercase word, apply standard lemmatization
            lemmatized_word = lemmatizer.lemmatize(word)
            
        # 6. Stopword Removal (Applied after lemmatization/categorization)
        if lemmatized_word and lemmatized_word not in stop_words:
            processed_tokens.append(lemmatized_word)
            
    return processed_tokens


In [19]:
def calculate_counts_and_clean(df):
    """Calculates all count features and preprocesses the essay column."""
    df_result = df.copy()

    # Apply the full preprocessing pipeline (Steps 5-9 equivalent)
    df_result['processed_tokens'] = df_result['essay'].apply(preprocess_essay)
    
    # 1. Word Count (Total words in raw text, similar to your mylist.count logic but for raw tokens)
    df_result['word_count'] = df_result['essay'].apply(lambda x: len(str(x).split()))

    # 2. Unique Words (Count of unique processed/lemmatized tokens)
    df_result['unique_words'] = df_result['processed_tokens'].apply(lambda tokens: len(set(tokens)))

    # 3. Stopword Count (Based on processed tokens used in your vectorization)
    # The stopword count in your reference code is based on the final lemmatized list (mylist). 
    # Here, we calculate it based on the words removed during preprocessing.
    def count_stopwords_in_raw(essay):
        if pd.isna(essay): return 0
        
        # 1. Apply selective lowercase and punctuation removal on raw text
        cased_tokens = [word if word.startswith('@') else word.lower() for word in str(essay).split()]
        joined_string = ' '.join(cased_tokens)
        clean_string = joined_string.translate(translator)
        clean_tokens = clean_string.split()
        
        # 2. Count stopwords BEFORE processing/categorization
        return sum(1 for word in clean_tokens if word in stop_words)

    df_result['stopword'] = df_result['essay'].apply(count_stopwords_in_raw)
    
    return df_result


In [20]:
def create_feature_matrix(df_result):
    """
    Implements Task 10: Finds ALL unique processed words from score 12 essays 
    (the 'terms' in your code) and creates feature columns.
    """
    
    # 1. Filter essays with the target score (equivalent to d1t, d2t, d3t in your code)
    high_score_tokens = df_result[df_result['domain1_score'] == SCORE_FOR_COMMON_WORDS]['processed_tokens']

    # 2. Aggregate all tokens (equivalent to finding the union of all word sets)
    all_high_score_tokens = [token for sublist in high_score_tokens for token in sublist]

    # 3. Find ALL unique words (the 'terms' set in your code)
    word_counts = Counter(all_high_score_tokens)
    common_words = list(word_counts.keys())
    
    print(f"Defining features based on ALL {len(common_words)} unique words from essays with score {SCORE_FOR_COMMON_WORDS}.")
    
    # 4. Create new feature columns by counting these words in EVERY essay
    for word in common_words:
        # This is the equivalent of your vector_d(mylist) function applied to all documents
        df_result[f'feature_{word}'] = df_result['processed_tokens'].apply(lambda tokens: tokens.count(word))
        
    return df_result


In [21]:

# --- Main Execution Block ---
try:
    df = pd.read_excel(INPUT_FILE)
    print(f"Successfully loaded data from: {INPUT_FILE}")

    # 1. Calculate all counts and clean/tokenize the text
    df_with_counts = calculate_counts_and_clean(df)

    # 2. Create the common word features (Task 10)
    df_final = create_feature_matrix(df_with_counts)
    
    # 3. Finalize columns and save
    final_columns = [
        'document_number', 
        'domain1_score', 
        'word_count', 
        'stopword', 
        'unique_words'
    ]
    # Add all feature columns
    final_columns.extend([col for col in df_final.columns if col.startswith('feature_')])
    
    # Select only the required columns, dropping the original essay and temporary tokens
    df_output = df_final[final_columns].copy() 

    print(f"Saving final features to: {OUTPUT_FILE}")
    df_output.to_excel(OUTPUT_FILE, index=False)
    
    print("\n✅ Execution Complete.")
    print(f"Final DataFrame saved to '{OUTPUT_FILE}' with {len(df_output.columns)} columns.")

except FileNotFoundError:
    print(f"\n❌ Error: The input file '{INPUT_FILE}' was not found.")
    print("Please make sure the file is in the correct location or upload it.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")
    print(f"Details: {e}")

Successfully loaded data from: raw_data_essay_set1.xlsx
Defining features based on ALL 1218 unique words from essays with score 12.


C:\Users\bhatt\AppData\Local\Temp\ipykernel_18036\1792307566.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result[f'feature_{word}'] = df_result['processed_tokens'].apply(lambda tokens: tokens.count(word))
C:\Users\bhatt\AppData\Local\Temp\ipykernel_18036\1792307566.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result[f'feature_{word}'] = df_result['processed_tokens'].apply(lambda tokens: tokens.count(word))
C:\Users\bhatt\AppData\Local\Temp\ipykernel_18036\1792307566.py:22: PerformanceWarning: DataFrame is h

Saving final features to: essay_dataset.xlsx

✅ Execution Complete.
Final DataFrame saved to 'essay_dataset.xlsx' with 1223 columns.
